In [25]:
import gzip
import ujson
import pandas as pd
import sklearn as sk
import numpy as np

In [7]:
filedict = []
with gzip.open("data/yelp_train_academic_dataset_business.json.gz",'rb') as f:
    for line in f:
        tempdict= ujson.loads(line)
        filedict.append(tempdict)

In [11]:
df = pd.DataFrame(filedict)

## City_model

In [20]:
city_df = df[['city','stars']]

In [32]:
gcity = city_df.groupby("city").agg({'stars':np.mean}).reset_index().head()

In [33]:
gcity.head()

,city,stars
0,Ahwatukee,3.687500
1,Anthem,3.781818
2,Apache Junction,3.637500
3,Arcadia,5.000000
4,Atlanta,3.500000


In [44]:
float(gcity[gcity.city=="Atlanta"].stars)

3.5

In [51]:
class City_model(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self):
        pass
    def fit(self, city_df):
        self.city_grouped = df.groupby('city').agg({'stars':np.mean}).reset_index()
    def predict(self, name):
        return float(self.city_grouped[self.city_grouped.city==name].stars)

In [52]:
city_model = City_model()

In [53]:
city_model.fit(city_df)

In [55]:
city_model.predict("Atlanta")

3.5

In [56]:
from sklearn.externals import joblib

In [59]:
joblib.dump(city_model, 'model/city/city_model.pkl') 

['model/city/city_model.pkl',
 'model/city/city_model.pkl_01.npy',
 'model/city/city_model.pkl_02.npy',
 'model/city/city_model.pkl_03.npy',
 'model/city/city_model.pkl_04.npy',
 'model/city/city_model.pkl_05.npy',
 'model/city/city_model.pkl_06.npy',
 'model/city/city_model.pkl_07.npy',
 'model/city/city_model.pkl_08.npy']